<a href="https://colab.research.google.com/github/HammadN98/pytorch/blob/main/04_pytorch_dados_personalizados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04. Pytorch para datasets proprios

## Cada tipo tem um "pytorch" proprio

* Texto - torchtext
* Audio - torchaudio
* Sistema de Recomendacao - torchrec
* Visao computacional - torchvision

Cada tipo tem uma biblioteca para funcoes de carregamento de dados e funcoes customizadas para o mesmo

## 0. Importando Pytorch e fazendo o selecionador de dispositivo

In [1]:
import torch
from torch import nn

torch.__version__

'2.3.1+cu121'

In [4]:
#Diagnostico de dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'